# 2D SSN Model

1. Imports

In [1]:
import jax.numpy as np
import jax
import matplotlib.pyplot as plt
import time, os, json
import pandas as pd
from scipy import stats 
from tqdm import tqdm
import seaborn as sns
from jax import random
from jax.config import config 
import pdb
import optax

#config.update('jax_debug_nans', True)
from SSN_classes_jax import SSN2DTopoV1_AMPAGABA_ONOFF
from util import GaborFilter, BW_Grating, find_A, create_gabor_filters, create_gratings

# MODEL TRAINING

## --> INCLUDE NUMBER OF TRIALS

In [2]:
def sigmoid(x):
    return 1/(1+np.exp(x))

def binary_loss(n, x):
    return - (n*np.log(x) + (1-n)*np.log(1-x))

def model(opt_pars, ssn_pars, grid_pars, conn_pars, train_data, labels, filter_pars,  **conv_pars):
    
    signs=np.array([[1, -1], [1, -1]])
    
    J_2x2 =np.exp(opt_pars['logJ_2x2'])*signs
    s_2x2 = np.exp(opt_pars['logs_2x2'])
    
    total_loss=0
    
    for i in range(len(train_data)):
        #Initialise network
        ssn=SSN2DTopoV1_AMPAGABA_ONOFF(ssn_pars=ssn_pars, grid_pars=grid_pars, conn_pars=conn_pars, filter_pars=filter_pars, J_2x2=J_2x2, s_2x2=s_2x2)
                                                   
        #Apply Gabor filters to stimuli
        output_ref=np.matmul(ssn.gabor_filters, train_data[i, 0].ravel())*ssn.A
        output_target=np.matmul(ssn.gabor_filters, train_data[i,1].ravel())*ssn.A

        #Rectify output
        SSN_input_ref=np.maximum(0, output_ref)
        SSN_input_target=np.maximum(0, output_target)
        
        #Input to SSN
        r_init = np.zeros(SSN_input_ref.shape[0])
        
        fp_ref, _ = ssn.fixed_point_r(SSN_input_ref, r_init=r_init, **conv_pars)
        x_ref = ssn.apply_bounding_box(fp_ref, size=3.2)

        fp_target, _ = ssn.fixed_point_r(SSN_input_target, r_init=r_init, **conv_pars)
        x_target = ssn.apply_bounding_box(fp_target, size=3.2)

        #Apply sigmoid function - combine ref and target
        x = sigmoid( np.dot(opt_pars['w_sig'], (x_ref.ravel() - x_target.ravel())) + opt_pars['b_sig'])

        #Calculate binary cross entropy loss
        total_loss+=np.sum(binary_loss(labels[i], x))
        
    print('Loss {}'.format(total_loss))
   
    return total_loss
    

def train_SSN(opt_pars, ssn_pars, grid_pars, conn_pars, train_data, labels, filter_pars, conv_pars, epochs=1, eta=10e-4):
    
    #Initialize loss
    loss_per_epoch = []
    
    #Initialise optimizer
    optimizer = optax.adam(eta)
    opt_state = optimizer.init(opt_pars)
    
    for epoch in range(epochs):
        
        
        #compute loss and gradient 
        loss, grad =jax.value_and_grad(model)(opt_pars, ssn_pars, grid_pars, conn_pars, train_data, labels, filter_pars,  **conv_pars)
        
        print('Loss {} at epoch {}'.format(loss, epoch+1))
        
        #Apply SGD through Adam optimizer
        updates, opt_state = optimizer.update(grad, opt_state)
        opt_pars = optax.apply_updates(opt_pars, updates)
        
        loss_per_epoch.append(loss)
    
    #reparametize parameters
    signs=np.array([[1, -1], [1, -1]])    
    opt_pars['logJ_2x2'] = np.exp(opt_pars['logJ_2x2'])*signs
    opt_pars['logs_2x2'] = np.exp(opt_pars['logs_2x2'])
    
    return opt_pars, loss_per_epoch

1. Define parameters

In [7]:
#Convergence parameters

#Network parameters
class ssn_pars():
    n = 2
    k = 0.04
    tauE = 30 # in ms
    tauI = 10 # in ms
    psi = 0.774
    tau_s = np.array([5, 7, 100]) #in ms, AMPA, GABA, NMDA current decay time constants
    

#Grid parameters
class grid_pars():
    gridsize_Nx = 9 # grid-points across each edge # gives rise to dx = 0.8 mm
    gridsize_deg = 2 * 1.6 # edge length in degrees
    magnif_factor = 2  # mm/deg
    hyper_col = 0.8 # mm   
    sigma_RF = 0.4 # deg (visual angle)

# Caleb's params for the full (with local) model:
Js0 = [1.82650658, 0.68194475, 2.06815311, 0.5106321]
gE, gI = 0.57328625, 0.26144141

sigEE, sigIE = 0.2, 0.40
sigEI, sigII = .09, .09
conn_pars = dict(
    PERIODIC = False,
    p_local = [.4, 0.7], # [p_local_EE, p_local_IE],
    sigma_oris = 1000) # sigma_oris


make_J2x2 = lambda Jee, Jei, Jie, Jii: np.array([[Jee, -Jei], [Jie,  -Jii]]) * np.pi * ssn_pars.psi
J_2x2 = make_J2x2(*Js0)
s_2x2 = np.array([[sigEE, sigEI],[sigIE, sigII]])

#Positive reparameterization
signs=np.array([[1, -1], [1, -1]])
logJ_2x2 =np.log(J_2x2*signs)
logs_2x2 = np.log(s_2x2)


#Sigmoid parameters
N_neurons = 25
key = random.PRNGKey(10)
w_sig = random.normal(key, shape = (N_neurons,)) / np.sqrt(N_neurons)
b_sig = 0.0

#Gabor parameters 
sigma_g= 0.5
k = np.pi/(6*sigma_g)

#Optimization pars
opt_pars = dict(logJ_2x2 = logJ_2x2, logs_2x2 = logs_2x2, w_sig = w_sig, b_sig=b_sig)

#Parameters shared with stimuli
general_pars = dict(k=k , edge_deg=3.2,  degree_per_pixel=0.05)

#Parameters exclusive to Gabor filters
filter_pars = dict(sigma_g = sigma_g, conv_factor = grid_pars.magnif_factor)
filter_pars.update(general_pars) 

2. Create training data

In [11]:
#Stimuli parameters
stimuli_pars = dict(outer_radius=3, inner_radius=2.5, grating_contrast=0.99, snr = 0.9)
stimuli_pars.update(general_pars)

#Create gratings at given orientation and list of labels
gratings, labels = create_gratings(ref_ori=55, number=100, offset= 40, jitter_val=0, **stimuli_pars)

3. TRAINING!

In [38]:
conv_pars=dict(dt = 1, xtol = 1e-5, Tmax = 200, verbose=False, silent=True)

#model_test = model(opt_pars, ssn_pars, grid_pars, conn_pars, gratings, labels, filter_pars,  **conv_pars)
new_pars, loss= train_SSN(opt_pars, ssn_pars, grid_pars, conn_pars, gratings, labels, filter_pars, conv_pars, epochs=1)

Traced<ConcreteArray(301.4615173339844, dtype=float32)>with<JVPTrace(level=2/0)> with
  primal = DeviceArray(301.46152, dtype=float32)
  tangent = Traced<ShapedArray(float32[])>with<JaxprTrace(level=1/0)> with
    pval = (ShapedArray(float32[]), None)
    recipe = JaxprEqnRecipe(eqn_id=<object object at 0x7f89a840e9f0>, in_tracers=(Traced<ShapedArray(float32[324]):JaxprTrace(level=1/0)>,), out_tracer_refs=[<weakref at 0x7f89c80a4720; to 'JaxprTracer' at 0x7f89c80a4220>], out_avals=[ShapedArray(float32[])], primitive=xla_call, params={'device': None, 'backend': None, 'name': '_reduce_sum', 'donated_invars': (False,), 'inline': True, 'keep_unused': False, 'call_jaxpr': { lambda ; a:f32[324]. let b:f32[] = reduce_sum[axes=(0,)] a in (b,) }}, effects=set(), source_info=SourceInfo(traceback=<jaxlib.xla_extension.Traceback object at 0x7f897456e470>, name_stack=NameStack(stack=(Transform(name='jvp'),))))
Traced<ConcreteArray(300.1659851074219, dtype=float32)>with<JVPTrace(level=2/0)> with
  p

Traced<ConcreteArray(-1.0381773710250854, dtype=float32)>with<JVPTrace(level=2/0)> with
  primal = DeviceArray(-1.0381774, dtype=float32)
  tangent = Traced<ShapedArray(float32[])>with<JaxprTrace(level=1/0)> with
    pval = (ShapedArray(float32[]), None)
    recipe = JaxprEqnRecipe(eqn_id=<object object at 0x7f89a8367820>, in_tracers=(Traced<ShapedArray(float32[324]):JaxprTrace(level=1/0)>,), out_tracer_refs=[<weakref at 0x7f890dcd19a0; to 'JaxprTracer' at 0x7f890dcd1d60>], out_avals=[ShapedArray(float32[])], primitive=xla_call, params={'device': None, 'backend': None, 'name': '_reduce_sum', 'donated_invars': (False,), 'inline': True, 'keep_unused': False, 'call_jaxpr': { lambda ; a:f32[324]. let b:f32[] = reduce_sum[axes=(0,)] a in (b,) }}, effects=set(), source_info=SourceInfo(traceback=<jaxlib.xla_extension.Traceback object at 0x7f89a040b4f0>, name_stack=NameStack(stack=(Transform(name='jvp'),))))
Traced<ConcreteArray(-0.9167177081108093, dtype=float32)>with<JVPTrace(level=2/0)> wit

Traced<ConcreteArray(0.16371923685073853, dtype=float32)>with<JVPTrace(level=2/0)> with
  primal = DeviceArray(0.16371924, dtype=float32)
  tangent = Traced<ShapedArray(float32[])>with<JaxprTrace(level=1/0)> with
    pval = (ShapedArray(float32[]), None)
    recipe = JaxprEqnRecipe(eqn_id=<object object at 0x7f89a866a130>, in_tracers=(Traced<ShapedArray(float32[324]):JaxprTrace(level=1/0)>,), out_tracer_refs=[<weakref at 0x7f8984549040; to 'JaxprTracer' at 0x7f8984549130>], out_avals=[ShapedArray(float32[])], primitive=xla_call, params={'device': None, 'backend': None, 'name': '_reduce_sum', 'donated_invars': (False,), 'inline': True, 'keep_unused': False, 'call_jaxpr': { lambda ; a:f32[324]. let b:f32[] = reduce_sum[axes=(0,)] a in (b,) }}, effects=set(), source_info=SourceInfo(traceback=<jaxlib.xla_extension.Traceback object at 0x7f89a85a7a70>, name_stack=NameStack(stack=(Transform(name='jvp'),))))
Traced<ConcreteArray(0.15996681153774261, dtype=float32)>with<JVPTrace(level=2/0)> wit

Traced<ConcreteArray(0.06253541260957718, dtype=float32)>with<JVPTrace(level=2/0)> with
  primal = DeviceArray(0.06253541, dtype=float32)
  tangent = Traced<ShapedArray(float32[])>with<JaxprTrace(level=1/0)> with
    pval = (ShapedArray(float32[]), None)
    recipe = JaxprEqnRecipe(eqn_id=<object object at 0x7f89c824fc50>, in_tracers=(Traced<ShapedArray(float32[324]):JaxprTrace(level=1/0)>,), out_tracer_refs=[<weakref at 0x7f89a8225360; to 'JaxprTracer' at 0x7f89a8225040>], out_avals=[ShapedArray(float32[])], primitive=xla_call, params={'device': None, 'backend': None, 'name': '_reduce_sum', 'donated_invars': (False,), 'inline': True, 'keep_unused': False, 'call_jaxpr': { lambda ; a:f32[324]. let b:f32[] = reduce_sum[axes=(0,)] a in (b,) }}, effects=set(), source_info=SourceInfo(traceback=<jaxlib.xla_extension.Traceback object at 0x7f89a046e670>, name_stack=NameStack(stack=(Transform(name='jvp'),))))
Traced<ConcreteArray(0.060808055102825165, dtype=float32)>with<JVPTrace(level=2/0)> wi

Traced<ConcreteArray(0.02530558407306671, dtype=float32)>with<JVPTrace(level=2/0)> with
  primal = DeviceArray(0.02530558, dtype=float32)
  tangent = Traced<ShapedArray(float32[])>with<JaxprTrace(level=1/0)> with
    pval = (ShapedArray(float32[]), None)
    recipe = JaxprEqnRecipe(eqn_id=<object object at 0x7f89ec6eb130>, in_tracers=(Traced<ShapedArray(float32[324]):JaxprTrace(level=1/0)>,), out_tracer_refs=[<weakref at 0x7f890c10c0e0; to 'JaxprTracer' at 0x7f890c10c220>], out_avals=[ShapedArray(float32[])], primitive=xla_call, params={'device': None, 'backend': None, 'name': '_reduce_sum', 'donated_invars': (False,), 'inline': True, 'keep_unused': False, 'call_jaxpr': { lambda ; a:f32[324]. let b:f32[] = reduce_sum[axes=(0,)] a in (b,) }}, effects=set(), source_info=SourceInfo(traceback=<jaxlib.xla_extension.Traceback object at 0x7f89a8536870>, name_stack=NameStack(stack=(Transform(name='jvp'),))))
Traced<ConcreteArray(0.02463381551206112, dtype=float32)>with<JVPTrace(level=2/0)> wit

...

Traced<ConcreteArray(-0.22811883687973022, dtype=float32)>with<JVPTrace(level=2/0)> with
  primal = DeviceArray(-0.22811884, dtype=float32)
  tangent = Traced<ShapedArray(float32[])>with<JaxprTrace(level=1/0)> with
    pval = (ShapedArray(float32[]), None)
    recipe = JaxprEqnRecipe(eqn_id=<object object at 0x7f89a062f420>, in_tracers=(Traced<ShapedArray(float32[324]):JaxprTrace(level=1/0)>,), out_tracer_refs=[<weakref at 0x7f890d5be720; to 'JaxprTracer' at 0x7f890d5be4f0>], out_avals=[ShapedArray(float32[])], primitive=xla_call, params={'device': None, 'backend': None, 'name': '_reduce_sum', 'donated_invars': (False,), 'inline': True, 'keep_unused': False, 'call_jaxpr': { lambda ; a:f32[324]. let b:f32[] = reduce_sum[axes=(0,)] a in (b,) }}, effects=set(), source_info=SourceInfo(traceback=<jaxlib.xla_extension.Traceback object at 0x7f890d5b1130>, name_stack=NameStack(stack=(Transform(name='jvp'),))))
Traced<ConcreteArray(-0.1631789207458496, dtype=float32)>with<JVPTrace(level=2/0)> w

Traced<ConcreteArray(0.20413771271705627, dtype=float32)>with<JVPTrace(level=2/0)> with
  primal = DeviceArray(0.20413771, dtype=float32)
  tangent = Traced<ShapedArray(float32[])>with<JaxprTrace(level=1/0)> with
    pval = (ShapedArray(float32[]), None)
    recipe = JaxprEqnRecipe(eqn_id=<object object at 0x7f88ff03e3d0>, in_tracers=(Traced<ShapedArray(float32[324]):JaxprTrace(level=1/0)>,), out_tracer_refs=[<weakref at 0x7f89a0628680; to 'JaxprTracer' at 0x7f88ff0660e0>], out_avals=[ShapedArray(float32[])], primitive=xla_call, params={'device': None, 'backend': None, 'name': '_reduce_sum', 'donated_invars': (False,), 'inline': True, 'keep_unused': False, 'call_jaxpr': { lambda ; a:f32[324]. let b:f32[] = reduce_sum[axes=(0,)] a in (b,) }}, effects=set(), source_info=SourceInfo(traceback=<jaxlib.xla_extension.Traceback object at 0x7f88ff0419b0>, name_stack=NameStack(stack=(Transform(name='jvp'),))))
Traced<ConcreteArray(0.19923925399780273, dtype=float32)>with<JVPTrace(level=2/0)> wit

Traced<ConcreteArray(0.08478015661239624, dtype=float32)>with<JVPTrace(level=2/0)> with
  primal = DeviceArray(0.08478016, dtype=float32)
  tangent = Traced<ShapedArray(float32[])>with<JaxprTrace(level=1/0)> with
    pval = (ShapedArray(float32[]), None)
    recipe = JaxprEqnRecipe(eqn_id=<object object at 0x7f89a035c8d0>, in_tracers=(Traced<ShapedArray(float32[324]):JaxprTrace(level=1/0)>,), out_tracer_refs=[<weakref at 0x7f890cb75220; to 'JaxprTracer' at 0x7f890cb75630>], out_avals=[ShapedArray(float32[])], primitive=xla_call, params={'device': None, 'backend': None, 'name': '_reduce_sum', 'donated_invars': (False,), 'inline': True, 'keep_unused': False, 'call_jaxpr': { lambda ; a:f32[324]. let b:f32[] = reduce_sum[axes=(0,)] a in (b,) }}, effects=set(), source_info=SourceInfo(traceback=<jaxlib.xla_extension.Traceback object at 0x7f890cb44970>, name_stack=NameStack(stack=(Transform(name='jvp'),))))
Traced<ConcreteArray(0.08259086310863495, dtype=float32)>with<JVPTrace(level=2/0)> wit

Traced<ConcreteArray(0.03472917526960373, dtype=float32)>with<JVPTrace(level=2/0)> with
  primal = DeviceArray(0.03472918, dtype=float32)
  tangent = Traced<ShapedArray(float32[])>with<JaxprTrace(level=1/0)> with
    pval = (ShapedArray(float32[]), None)
    recipe = JaxprEqnRecipe(eqn_id=<object object at 0x7f89a066cf50>, in_tracers=(Traced<ShapedArray(float32[324]):JaxprTrace(level=1/0)>,), out_tracer_refs=[<weakref at 0x7f890d390220; to 'JaxprTracer' at 0x7f890d3909f0>], out_avals=[ShapedArray(float32[])], primitive=xla_call, params={'device': None, 'backend': None, 'name': '_reduce_sum', 'donated_invars': (False,), 'inline': True, 'keep_unused': False, 'call_jaxpr': { lambda ; a:f32[324]. let b:f32[] = reduce_sum[axes=(0,)] a in (b,) }}, effects=set(), source_info=SourceInfo(traceback=<jaxlib.xla_extension.Traceback object at 0x7f890d3a9730>, name_stack=NameStack(stack=(Transform(name='jvp'),))))
Traced<ConcreteArray(0.03387674316763878, dtype=float32)>with<JVPTrace(level=2/0)> wit

Loss 1.358145833015442 at epoch 1


In [15]:
ssn=SSN2DTopoV1_AMPAGABA_ONOFF(ssn_pars=ssn_pars, grid_pars=grid_pars, conn_pars=conn_pars, filter_pars=filter_pars, J_2x2=J_2x2, s_2x2=s_2x2)

#Apply Gabor filters to stimuli
output_ref=np.matmul(ssn.gabor_filters, gratings[1, 0].ravel())*ssn.A
output_target=np.matmul(ssn.gabor_filters, gratings[1,1].ravel())*ssn.A

#Rectify output
SSN_input_ref=np.maximum(0, output_ref)
SSN_input_target=np.maximum(0, output_target)

#Input to SSN
r_init = np.zeros(SSN_input_ref.shape[0])

fp_ref, _ = ssn.fixed_point_r(SSN_input_ref, r_init=r_init, **conv_pars)

#fp_target, _ = ssn.fixed_point_r(SSN_input_target, r_init=r_init, **conv_pars)


KeyboardInterrupt



In [8]:
np.sum(ssn.W)

DeviceArray(2128.79824252, dtype=float64)